In [ ]:
from pyabc import ABCSMC, Distribution, RV
import scipy as sp
from tempfile import gettempdir
import os

In [ ]:
def model(parameter):
    return {"data": parameter["mean"] + sp.randn()}

prior = Distribution(mean=RV("uniform", 0, 5))

def distance(x, y):
    return abs(x["data"] - y["data"])

db = "sqlite:///" + os.path.join(gettempdir(), "test.db")

In [ ]:
abc = ABCSMC(model, prior, distance)
run_id = abc.new(db, {"data": 2.5})
print("Run ID:", run_id)

In [ ]:
history = abc.run(minimum_epsilon=.1, max_nr_populations=3)

In [ ]:
history.n_populations

In [ ]:
abc_continued = ABCSMC(model, prior, distance)

In [ ]:
abc_continued.load(db, run_id)

In [ ]:
abc_continued.run(minimum_epsilon=.1, max_nr_populations=1)

In [ ]:
abc_continued.history.n_populations